In [1]:
import numpy as np
import os
import time
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from keras.models import load_model
import audio_processor as ap

c:\users\yizong\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\yizong\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import glob

In [3]:
glob.glob("mini_test/*.mp3")

[]

In [4]:
for i in glob.glob("data/mini_test/*.mp3"):
    print (i)

data/mini_test\Chopin - Andante spinato.mp3
data/mini_test\Chopin - Ballade in A flat major op.47 no.3.mp3
data/mini_test\Chopin - Ballade in F major op.38 No. 2.mp3
data/mini_test\Chopin - Ballade in G minor, op.23 no.1.mp3
data/mini_test\ChopinFuneralMarch.mp3
data/mini_test\ChopinFuneralMarch2.mp3
data/mini_test\ChopinHeroicPolonaise.mp3
data/mini_test\ChopinMinuteWaltz.mp3
data/mini_test\ChopinNocturneinGminor.mp3
data/mini_test\ChopinPianoNocturninEbm.mp3
data/mini_test\ChopinRomanticPiano.mp3
data/mini_test\Kevin Kern - Because I Love You.mp3
data/mini_test\Kevin Kern - dance.mp3
data/mini_test\Kevin Kern - go the green.mp3
data/mini_test\Kevin Kern - Le Jardin.mp3
data/mini_test\Kevin Kern - secret.mp3
data/mini_test\Kevin Kern - sun.mp3
data/mini_test\Kevin Kern - The Touch Of Love.mp3
data/mini_test\Kevin Kern - wave.mp3
data/mini_test\Kevin Kern - We Should Waltz.mp3


In [5]:
melgrams = np.zeros((0, 1, 96, 1366))
for i in glob.glob("data/mini_test/*.mp3"):
    melgram = ap.compute_melgram(i)
    melgrams = np.concatenate((melgrams, melgram), axis=0)

In [6]:
melgrams = melgrams.transpose(0,2,3,1)


In [9]:
num_of_classes = 2
num_of_samples = 20
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:10]=0
labels[10:]=1
names = ['Chopin','Other']
Y = np_utils.to_categorical(labels, num_of_classes)
x,y = shuffle(melgrams,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [10]:
trans_model = load_model('transfer_model.h5')

c:\users\yizong\appdata\local\programs\python\python36\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
trans_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 96, 1366, 1)       0         
_________________________________________________________________
bn_0_freq (BatchNormalizatio (None, 96, 1366, 1)       384       
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 1366, 32)      320       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 96, 1366, 32)      128       
_________________________________________________________________
elu_46 (ELU)                 (None, 96, 1366, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 341, 32)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 341, 64)       18496     
__________

In [12]:
from keras import backend as K

In [20]:
num_of_classes = 2
num_of_samples = 20
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:10]=0
labels[10:]=1
names = ['Chopin','Other']
Y = np_utils.to_categorical(labels, num_of_classes)
x,y = shuffle(melgrams,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
last_layear = trans_model.get_layer('pool3').output
flat = Flatten(name='flatten')(last_layear)
out = Dense(num_of_classes, activation='softmax', name='output_layer')(flat)
custom_model = Model(inputs=melgrams_input,outputs= out)

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_10:0", shape=(?, 96, 1366, 1), dtype=float32) at layer "input_10". The following previous layers were accessed without issue: []

In [112]:
trans_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 96, 1366, 1)       0         
_________________________________________________________________
bn_0_freq (BatchNormalizatio (None, 96, 1366, 1)       384       
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 1366, 32)      320       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 96, 1366, 32)      128       
_________________________________________________________________
elu_46 (ELU)                 (None, 96, 1366, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 341, 32)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 341, 64)       18496     
__________

In [21]:
melgrams_input = Input(shape=(96,1366,1))

In [22]:
melgrams_input

<tf.Tensor 'input_2:0' shape=(?, 96, 1366, 1) dtype=float32>

In [23]:
last_layear = trans_model.get_layer('pool5').output
flat = Flatten(name='flatten')(last_layear)
out = Dense(num_of_classes, activation='softmax', name='output_layer')(flat)
custom_model = Model(melgrams_input,outputs= out)

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_10:0", shape=(?, 96, 1366, 1), dtype=float32) at layer "input_10". The following previous layers were accessed without issue: []